In [1]:
# DS4200
# Final Project
# Data Cleaning Portion

In [117]:
import pandas as pd
import numpy as np

In [3]:
# Read in the files
resort_df = pd.read_csv("H1.csv")
city_df = pd.read_csv("H2.csv")

In [109]:
# Append the 2 datasets
# Create a column to differentiate the booking type (city, hotel)
resort_df["HotelType"] = "Resort"
city_df["HotelType"] = "City"

# Now conduct the merge
bookings_df = pd.concat((resort_df, city_df), axis=0)

# Filter the data down to 3 countries --> Spain, France, Italy
#country_lst = list("ESP", "FRA", "ITA")
bookings_df = bookings_df.loc[bookings_df["Country"].isin(["ESP", "FRA", "ITA"])]
print(set(bookings_df["Country"]))

{'ITA', 'FRA', 'ESP'}


In [143]:
# Remove whitespace to find NULL values
# and convert them to represent the NaN data type
bookings_df = bookings_df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
bookings_df.replace("NULL", np.nan, inplace=True)

# Find the amount of NaN rows in the dataset 
# and remove them
null_rows = bookings_df[bookings_df.isnull().all(axis=1)]
#print(null_rows)  #32

null_idx = null_rows.index.to_list()
bookings_df = bookings_df.drop(index=null_idx)

# Find which column's have the highest prevalency
# of null values
for col in list(bookings_df.columns):
    null_amt = bookings_df[col].isnull().sum()
    pct = (null_amt / len(bookings_df)) * 100
    print(f"{col}: {round(pct, 3)}%")

IsCanceled: 0.0%
LeadTime: 0.0%
ArrivalDateYear: 0.0%
ArrivalDateMonth: 0.0%
ArrivalDateWeekNumber: 0.0%
ArrivalDateDayOfMonth: 0.0%
StaysInWeekendNights: 0.0%
StaysInWeekNights: 0.0%
Adults: 0.0%
Children: 0.0%
Babies: 0.0%
Meal: 0.0%
Country: 0.0%
MarketSegment: 0.0%
DistributionChannel: 0.0%
IsRepeatedGuest: 0.0%
PreviousCancellations: 0.0%
PreviousBookingsNotCanceled: 0.0%
ReservedRoomType: 0.0%
AssignedRoomType: 0.0%
BookingChanges: 0.0%
DepositType: 0.0%
Agent: 6.752%
DaysInWaitingList: 0.0%
CustomerType: 0.0%
ADR: 0.0%
RequiredCarParkingSpaces: 0.0%
TotalOfSpecialRequests: 0.0%
ReservationStatus: 0.0%
ReservationStatusDate: 0.0%
HotelType: 0.0%


In [181]:
# Convert columns to Strings
string_cols = ["ArrivalDateMonth", "Meal", "Country", "MarketSegment",
               "DistributionChannel", "ReservedRoomType", "AssignedRoomType",
               "DepositType", "Agent", "CustomerType", "ReservationStatus", "HotelType"]
for col in string_cols:
    bookings_df[col] = bookings_df[col].astype(str)

# Convert column to Datetime objects
bookings_df["ReservationStatusDate"] = pd.to_datetime(bookings_df["ReservationStatusDate"])

In [187]:
# Based on the output above, it looks like the
# `Company` column is useless due to the high 
# percentage of NaN values
#bookings_df = bookings_df.drop("Company", axis=1)

# Replace NaN values in the `Agent` column
# with the most common
bookings_df["Agent"] = bookings_df["Agent"].replace("nan", np.nan)

mode_agent = bookings_df["Agent"].mode().iloc[0]
bookings_df["Agent"] = bookings_df["Agent"].fillna(mode_agent)

In [ ]:
# create demographic features to focus on customer demographics and cancellation behavior 

# Create family demographic features
bookings_df['FamilySize'] = bookings_df['Adults'] + bookings_df['Children'] + bookings_df['Babies']
bookings_df['HasChildren'] = ((bookings_df['Children'] > 0) | (bookings_df['Babies'] > 0)).astype(int)

# Create demographic segments
conditions = [
    (bookings_df['Adults'] == 1) & (bookings_df['Children'] == 0) & (bookings_df['Babies'] == 0),
    (bookings_df['Adults'] == 2) & (bookings_df['Children'] == 0) & (bookings_df['Babies'] == 0),
    (bookings_df['HasChildren'] == 1)
]
choices = ['Solo', 'Couple', 'Family']
bookings_df['TravelerType'] = np.select(conditions, choices, default='Group')

In [ ]:
# creates season based on arrival month for context
season_map = {
    'January': 'Winter', 'February': 'Winter', 'March': 'Spring',
    'April': 'Spring', 'May': 'Spring', 'June': 'Summer',
    'July': 'Summer', 'August': 'Summer', 'September': 'Fall',
    'October': 'Fall', 'November': 'Fall', 'December': 'Winter'
}
bookings_df['Season'] = bookings_df['ArrivalDateMonth'].map(season_map)

In [ ]:
# creats business vs. leisure indication
# assumes that if in city and duration is short, then for a business trip (assumption) -> else unknown
conditions = [
    (bookings_df['HotelType'] == 'City') & (bookings_df['StaysInWeekNights'] >= 2) & (bookings_df['StaysInWeekendNights'] <= 1),
    (bookings_df['HotelType'] == 'Resort') | (bookings_df['StaysInWeekendNights'] >= 1)
]
choices = ['Business', 'Leisure']
bookings_df['TravelPurpose'] = np.select(conditions, choices, default='Unknown')

In [ ]:
# for calculating total duration of time in hotel
# create stay duration feature
bookings_df['TotalStay'] = bookings_df['StaysInWeekendNights'] + bookings_df['StaysInWeekNights']

In [ ]:
# checking distribution across countries and hotel types to make sure we dont have outliers in our data set skewing anythin 
print(bookings_df.groupby(['Country', 'HotelType'])['IsCanceled'].count())
print(bookings_df.groupby(['Country', 'HotelType'])['IsCanceled'].mean())

In [ ]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# DATA PREP
# Create a datetime column for better plotting
bookings_df['date'] = pd.to_datetime(bookings_df['ArrivalDateYear'].astype(str) + '-' + 
                           bookings_df['ArrivalDateMonth'] + '-01')

# Function to calculate cancellation rates by country and guest type
def calculate_cancellation_by_country(df):
    # Group by country, date and repeated guest status
    result = df.groupby(['Country', 'date', 'IsRepeatedGuest']).agg({
        'IsCanceled': ['count', 'sum']
    }).reset_index()
    
    # Flatten the multi-index and rename columns
    result.columns = ['Country', 'date', 'IsRepeatedGuest', 'total_bookings', 'total_cancellations']
    
    # Calculate cancellation rate
    result['cancellation_rate'] = (result['total_cancellations'] / 
                                   result['total_bookings'] * 100)
    return result

# Calculate stats
cancellation_stats = calculate_cancellation_by_country(bookings_df)

# Create a figure with subplots - one row for each country
fig = make_subplots(
    rows=3, 
    cols=1,
    shared_xaxes=True,
    subplot_titles=("Spain (ESP)", "France (FRA)", "Italy (ITA)"),
    vertical_spacing=0.15  # Increased spacing between subplots
)

# Countries with specific colors as requested
countries = ['ESP', 'FRA', 'ITA']
country_colors = {'ESP': 'red', 'FRA': 'blue', 'ITA': 'green'}

# Add traces for each country in separate subplots
for i, country in enumerate(countries, 1):
    # Get data for new guests (IsRepeatedGuest = 0)
    new_data = cancellation_stats[
        (cancellation_stats['Country'] == country) & 
        (cancellation_stats['IsRepeatedGuest'] == 0)
    ]
    
    # Get data for repeated guests (IsRepeatedGuest = 1)
    repeated_data = cancellation_stats[
        (cancellation_stats['Country'] == country) & 
        (cancellation_stats['IsRepeatedGuest'] == 1)
    ]
    
    # Add trace for new guests - solid line
    fig.add_trace(
        go.Scatter(
            x=new_data['date'],
            y=new_data['cancellation_rate'],
            name=f'New Guests ({country})' if i == 1 else None,
            showlegend=i==1,
            mode='lines+markers',
            line=dict(color=country_colors[country], width=2, dash='solid'),  # Solid line
            marker=dict(size=6),
            hovertemplate='Date: %{x}<br>Cancellation Rate: %{y:.1f}%<extra></extra>'
        ),
        row=i, col=1
    )
    
    # Add trace for repeated guests - dashed line as requested
    fig.add_trace(
        go.Scatter(
            x=repeated_data['date'],
            y=repeated_data['cancellation_rate'],
            name=f'Repeated Guests ({country})' if i == 1 else None,
            showlegend=i==1,
            mode='lines+markers',
            line=dict(color=country_colors[country], width=2, dash='dash'),  # Dashed line 
            marker=dict(size=6),
            hovertemplate='Date: %{x}<br>Cancellation Rate: %{y:.1f}%<extra></extra>'
        ),
        row=i, col=1
    )
    
    # Update y-axis label
    fig.update_yaxes(title_text="Cancellation Rate (%)", row=i, col=1, range=[0, 100])

# Move filter buttons to more prominent position
fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=3, label="3m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all", label="All")
        ]),
        y=1.1,  # Position higher up
        x=0.05,
        bgcolor='rgba(150, 200, 250, 0.4)',  # Light blue background
        font=dict(size=12)
    ),
    row=3, col=1
)

# Update shared x-axis title
fig.update_xaxes(title_text="Date", row=3, col=1)

# Fix squished layout with more spacing
fig.update_layout(
    title=dict(
        text='Hotel Booking Cancellation Rates: New vs Repeated Guests by Country',
        x=0.5,
        xanchor='center',
        y=0.97,  # Move title up
        font=dict(size=20)  # Larger title font
    ),
    hovermode='x unified',
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.08,  # Move legend up for more space
        xanchor="center",
        x=0.5,
        font=dict(size=14),
        bgcolor='rgba(255, 255, 255, 0.8)'
    ),
    template='plotly_white',
    height=900,  # Increased height for better spacing
    margin=dict(l=80, r=80, t=150, b=100),  # Increased top margin
)

# Show plot
fig.show()